# Clean Playlist Generator

This notebook will take the saved songs from your spotify library and create a playlist with just the clean ones!

### Setup

In [39]:
import sys
import spotipy
import spotipy.util as util
import requests
import json
import pandas as pd
import numpy as np

# Declare API access keys and request information
scope = 'user-library-read playlist-modify-public'
username = ''
client_id = ''
client_secret = ''
redirect_uri = 'http://localhost/'

token = util.prompt_for_user_token(username,scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=fca05df727e74e95952546cbc88a86c0&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=playlist-modify-public+user-library-read in your browser




Enter the URL you were redirected to:  http://localhost/?code=AQBm2UqJ_2DWbHfDrxaNC916vFj37tGEt_FXlQe8OQHMnHqpAGaHoqq8DSX4F1QZgpNBBwMA8-nU-HPx5HpKyL-9zOr5Z64ETeUmBa2vySPV3v4neIkA-CVWbzsG-nhegKvi-tQ0ASEHiCYn23K7k2ilHAdUSDTQ7CbYdQjIfQZZ5BDsKWYx7lCJb1SRUV7ayphXqxYYIUt_I2Y4fhtFAQ


### Get User's Songs

Get the total number of songs you have saved

In [3]:
nextOffset = 0
nextLimit = 50

results = sp.current_user_saved_tracks(limit=nextLimit, offset=nextOffset)
print('Total Tracks: {}'.format(results['total']))

Total Tracks: 2294


In [22]:
# Create DataFrame to store results
df_my_songs = pd.DataFrame(columns=['song_uri', 'explicit'])

# Add results of initial request to DataFrame
for item in results['items']:
    df_my_songs = df_my_songs.append({
        'song_uri': item['track']['id'],
        'explicit': item['track']['explicit']
    }, ignore_index=True)
nextOffset += nextLimit

# Continue requesting song and names for all songs
while (results['next'] != None): 
    results = sp.current_user_saved_tracks(limit=nextLimit, offset=nextOffset)
    for item in results['items']:
        df_my_songs = df_my_songs.append({
            'song_uri': item['track']['id'],
            'explicit': item['track']['explicit']
        }, ignore_index=True)
    nextOffset += nextLimit

In [25]:
df_my_songs.head()

,song_uri,explicit
0,6H4vsxNsm2NsV8pBAb5WnU,True
1,34gCuhDGsG4bRPIf9bb02f,False
2,7LEMGMfmJDd1iLsGD9eV5l,False
3,3cwXD9KKjanH5y3d4hNDGc,False
4,3A0C7Uhq5KYUExhmCIGLEb,False


### Filter out Explicit Songs

In [28]:
df_clean = df_my_songs.copy(deep=True)
df_clean = df_clean[df_clean.explicit == False]

See how many non-explicit songs do we have

In [32]:
df_clean.shape

(1288, 2)

### Add songs to playlist

Create a playlist

In [53]:
userid = sp.current_user()['id']
playlist = sp.user_playlist_create(userid, 'Clean songs')
playlistid = playlist['id']

Add tracks to playlist

In [55]:
uris_to_add = df_clean.song_uri.values

start_ndx = 0
end_ndx = 100

while(end_ndx < len(uris_to_add)):
    print('Adding {} to {}'.format(start_ndx, end_ndx))
    sp.user_playlist_add_tracks(userid, playlistid, uris_to_add[start_ndx:end_ndx])
    start_ndx = end_ndx
    end_ndx = end_ndx + 100

print('Adding {} to end'.format(start_ndx))
sp.user_playlist_add_tracks(userid, playlistid, uris_to_add[start_ndx:])

Adding 0 to 100
Adding 100 to 200
Adding 200 to 300
Adding 300 to 400
Adding 400 to 500
Adding 500 to 600
Adding 600 to 700
Adding 700 to 800
Adding 800 to 900
Adding 900 to 1000
Adding 1000 to 1100
Adding 1100 to 1200
Adding 1200 to end


{'snapshot_id': 'MTQsODY1ZmM3NTcxNzdiMTAyNDViZGY2ZjAwYmIyMTkxZDc0M2QxYzgyNg=='}